In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from result_saver import SaverProvider
provider = SaverProvider()

In [ ]:
import numpy as np
from Scratch import metadata_loader

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]=="ibm_nazca"]

md = md[:2]

md

In [ ]:
memories = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()

memories

In [ ]:
import pickle
from soft_info import get_repcode_layout, get_KDEs, get_counts
from qiskit_qec.circuits import RepetitionCodeCircuit
from qiskit_qec.decoders import UnionFindDecoder

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend("ibm_sherbrooke"), _is_hex=True)

kde_dict, scaler_dict = get_KDEs(provider, 'ibm_sherbrooke', layout, bandwidths=0.2, plot=False)
print(kde_dict)

#code = RepetitionCodeCircuit(max_distance, max_distance)
#decoder = UnionFindDecoder(code)

with open('decoder.pkl', 'rb') as f:
    decoder = pickle.load(f)

In [ ]:
from soft_info import soft_reweight, get_counts
from tqdm import tqdm

memory = memories['mmr_log_0']

logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))):    
    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    reweighted_graph = soft_reweight(decoder, IQ_data, kde_dict, scaler_dict, layout)
    decoder.graph = reweighted_graph

    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    print("\nmeas_str:", count_key)
    print("flipped_qubit_dict", flipped_qubit_dict)
    print("logical:", logical, "\n")
    if logical[0] == 1:
        print("------------ Wrong logical! ------------\n")

    logical_counts[f"{logical[0]}"] += 1

print( "\nLogical_counts:", logical_counts)


In [ ]:
from soft_info import soft_reweight, get_counts
from tqdm import tqdm

memory = memories['mmr_log_0']

logical_counts = {'0':0, '1':0}
for shot in tqdm(range(len(memory))):    
    IQ_data = memory[shot]

    counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, max_distance, verbose=False)
    count_key = next(iter(counts.keys()))
    
    #reweighted_graph = soft_reweight(decoder, IQ_data, kde_dict, scaler_dict, layout)
    #decoder.graph = reweighted_graph

    logical, flipped_qubit_dict = decoder.process(count_key, _return_err_str = True)
    print("\nmeas_str:", count_key)
    print("flipped_qubit_dict", flipped_qubit_dict)
    print("logical:", logical, "\n")
    if logical[0] == 1:
        print("------------ Wrong logical! ------------\n")

    logical_counts[f"{logical[0]}"] += 1

print( "\nLogical_counts:", logical_counts)
